# People Counting

Intelligent Systems Assignment #2

> Actual Time People passed through the door: 16

## Source Code

In [33]:
import cv2
import numpy as np

# ----------------------------
# CONFIGURATION
# ----------------------------
VIDEO_PATH = "../video/IS-PEOPLE-WALK.mov"   # your input video
OUTPUT_PATH = "../video/OUTPUT-PEOPLE-COUNT.mp4"      # output video file

# counting line and detection zones
COUNT_LINE_X = 370
LEFT_ZONE = COUNT_LINE_X - 18
RIGHT_ZONE = COUNT_LINE_X + 18

# detection thresholds
MIN_AREA = 700
MIN_HEIGHT = 55

# cooldowns to prevent overcounting
PER_ID_COOLDOWN = 15          # per person
GLOBAL_COOLDOWN = 45          # whole system (~1.5s @30fps)

# ----------------------------
# OPEN VIDEO
# ----------------------------
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise IOError("Could not open video")

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30

# ----------------------------
# OUTPUT VIDEO WRITER
# ----------------------------
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # codec for MP4
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps * 2, (w, h))

# ----------------------------
# BACKGROUND SUBTRACTOR
# ----------------------------
bg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=40)

# ----------------------------
# TRACKING DATA
# ----------------------------
next_id = 0
last_x = {}
last_side = {}
last_count_frame = {}

total_passes = 0
frame_idx = 0
last_global_count_frame = -9999

# ----------------------------
# MAIN LOOP
# ----------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1

    # 1. get foreground mask
    mask = bg.apply(frame)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)

    # 2. clean mask
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=2)
    mask = cv2.dilate(mask, k, iterations=2)

    # 3. find moving objects
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < MIN_AREA:
            continue

        x, y, bw, bh = cv2.boundingRect(cnt)
        if bh < MIN_HEIGHT:
            continue

        cx = x + bw // 2
        cy = y + bh // 2

        # 4. match blob to existing ID
        assigned_id = None
        for oid, lx in last_x.items():
            if abs(cx - lx) < 45:
                assigned_id = oid
                break

        if assigned_id is None:
            assigned_id = next_id
            next_id += 1
            last_side[assigned_id] = None
            last_count_frame[assigned_id] = -9999

        last_x[assigned_id] = cx

        # draw bounding box + id
        cv2.rectangle(frame, (x, y), (x + bw, y + bh), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {assigned_id}", (x, y - 4),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)

        # 5. determine current side
        if cx <= LEFT_ZONE:
            cur_side = "L"
        elif cx >= RIGHT_ZONE:
            cur_side = "R"
        else:
            cur_side = "M"

        prev_side = last_side.get(assigned_id, None)

        # 6. detect crossing
        crossed = (
            (prev_side == "L" and cur_side == "R") or
            (prev_side == "R" and cur_side == "L")
        )

        if crossed:
            per_id_ok = frame_idx - last_count_frame[assigned_id] > PER_ID_COOLDOWN
            global_ok = frame_idx - last_global_count_frame > GLOBAL_COOLDOWN

            if per_id_ok and global_ok:
                total_passes += 1
                last_count_frame[assigned_id] = frame_idx
                last_global_count_frame = frame_idx
                cv2.putText(frame, "PASS", (cx + 5, cy),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        if cur_side in ("L", "R"):
            last_side[assigned_id] = cur_side

    # 7. draw counting lines
    cv2.line(frame, (COUNT_LINE_X, 0), (COUNT_LINE_X, h), (255, 0, 0), 2)
    cv2.line(frame, (LEFT_ZONE, 0), (LEFT_ZONE, h), (100, 100, 255), 1)
    cv2.line(frame, (RIGHT_ZONE, 0), (RIGHT_ZONE, h), (100, 255, 100), 1)

    # 8. display total count
    cv2.putText(frame, f"Passes: {total_passes}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 3)
    cv2.putText(frame, f"Passes: {total_passes}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 1)

    # 9. write output video
    out.write(frame)

    # 10. show live view
    cv2.imshow("People Counter", frame)
    cv2.imshow("Mask", mask)

    if cv2.waitKey(1) & 0xFF == 27:
        break

# ----------------------------
# CLEANUP
# ----------------------------
cap.release()
out.release()
cv2.destroyAllWindows()

print("Total passes:", total_passes)
print(f"Output video saved as: {OUTPUT_PATH}")

Total passes: 16
Output video saved as: ../video/OUTPUT-PEOPLE-COUNT.mp4
